In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import threshold_local
from ipynb.fs.full.transform import *
import imutils

In [2]:
def edge_detection(img):
    if(len(img.shape)>2):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img
    edged = cv2.Canny(gray, 230, 255)
    return edged

In [3]:
def get_contours(edged_img):
    gray = cv2.cvtColor(edged_img, cv2.COLOR_BGR2GRAY)
    (cnts, _) = cv2.findContours(gray.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key = cv2.contourArea, reverse=True)
    for contour in cnts:
        perimetr = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02*perimetr, True)
        if(len(approx)==4):
            return approx

In [4]:
def perspective_transform_apl(orig,contours, ratio):
    ordered_pts = order_points(contours)
    transformed_img = four_point_transform(orig, ordered_pts*ratio)
    return transformed_img

In [5]:
def final_transform(IMG_PATH):
    image = cv2.imread(IMG_PATH)
    lower_white = np.array([100,100,100])
    upper_white = np.array([255,255,255])
    mask = cv2.inRange(image, lower_white, upper_white)
    img_white_only = cv2.bitwise_and(image,image, mask= mask)
    ratio = 1 
    orig = image.copy()
    contours = get_contours(img_white_only)
    awd = cv2.drawContours(image, [contours], -1, (0,255,0),2)
    
    transformed = perspective_transform_apl(orig, contours.reshape(4,2), ratio)
    return transformed